In [ ]:
import yaml
from datacreator import StockData,EconomyData,FewShot_Sentiment_Generator
from openbb_terminal.sdk import openbb
from langchain.embeddings import OpenAIEmbeddings
from scipy.spatial import distance
from serpapi import GoogleSearch
import logging
logging.disable()
# Open the YAML file and load its contents into a Python object
with open('apis.yaml', 'r') as file:
    yaml_data = yaml.load(file, Loader=yaml.FullLoader)
from langchain.vectorstores import FAISS
# # Convert the YAML data to a dictionary
data_dict = dict(yaml_data)
openbb.keys.av(key = data_dict['OPENBB']['ALPHA_VANTAGE_KEY'], persist = True)
openbb.keys.fmp(key = data_dict['OPENBB']['FINANCIALMODELLING_AND_PREP_KEY'],persist = True)
openbb.keys.polygon(key = data_dict['OPENBB']['POLYGON_KEY'],persist = True)
openbb.keys.finnhub(key = data_dict['OPENBB']['FINNHUB_KEY'],persist = True)
openbb.keys.fred(key = data_dict['OPENBB']['FRED_KEY'],persist = True)
SERP_API_KEY = data_dict['GOOGLESEARCH']

In [ ]:
import time
for stock in ['AAPL','TSLA','GOOG']:
    stock_data = StockData(stock)
    stock_data.initialize_folders()
    stock_data.ratio_file()
    stock_data.cash_file()
    stock_data.balance_file()
    stock_data.est_file()
    stock_data.fraud_file()
    stock_data.income_file()
    stock_data.analysis_file()
    stock_data.news_file()
    time.sleep(1)

In [ ]:
import yaml
from datacreator import StockData, EconomyData
from openbb_terminal.sdk import openbb


# Open the YAML file and load its contents into a Python object
with open("apis.yaml", "r") as file:
    yaml_data = yaml.load(file, Loader=yaml.FullLoader)

# Convert the YAML data to a dictionary
data_dict = dict(yaml_data)
openbb.keys.av(key=data_dict["OPENBB"]["ALPHA_VANTAGE_KEY"], persist=True)
openbb.keys.fmp(
    key=data_dict["OPENBB"]["FINANCIALMODELLING_AND_PREP_KEY"], persist=True
)
openbb.keys.polygon(key=data_dict["OPENBB"]["POLYGON_KEY"], persist=True)
openbb.keys.finnhub(key=data_dict["OPENBB"]["FINNHUB_KEY"], persist=True)
openbb.keys.fred(key=data_dict["OPENBB"]["FRED_KEY"], persist=True)
open_ai_params = {
    "max_tokens": 512,
    "openai_api_key": yaml_data["LLMS"]["OPENAI_API_KEY"],
}
stocks = yaml_data['STOCKS']

# for stock in stocks:
#     stock_data = StockData(stock)
#     stock_data.initialize_folders()
#     stock_data.analysis_file()
#     stock_data.ratio_file()
#     stock_data.cash_file()
#     stock_data.est_file()
#     stock_data.fraud_file()
#     stock_data.income_file()
#     stock_data.balance_file()
#     stock_data.news_file()


In [ ]:
class Sentiment_Generator:
    def __init__(self,embeddings):
        self.embeddings = embeddings
    
    def collect_news_files(self):
        main_file = []
        for stock in os.listdir('../ticker'):
            try:
                df = pd.read_csv(f'../ticker/{stock}/news/c_news.csv')
                df.insert(0,'ticker',stock)
                df = df[['ticker','headline','datetime']]
            except:
                pass
            main_file.append(df)
        self.document_file = pd.concat(main_file)
    
    def get_scores(self):
        docs = list(self.document_file['headline'])
        classifications = ['Strongly Positive','Positive','Neutral','Negative','Strongly Negative']
        ## Create a faiss vector database
        #import pdb;pdb.set_trace()
        faiss_classifications = FAISS.from_texts(classifications,self.embeddings)
        self.similarity_scores = []
        for x in docs:
            #import pdb;pdb.set_trace()
            self.similarity_scores.append(faiss_classifications.similarity_search_with_score(x,k = 1)[0][0].page_content)
        self.document_file['similarity'] = self.similarity_scores
        
    def save_scores(self):
        self.document_file.to_csv('output/sentiment_scores.csv')
        

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key = yaml_data["LLMS"]["OPENAI_API_KEY"])
sentiment_generation = Sentiment_Generator(embeddings)
sentiment_generation.collect_news_files()
sentiment_generation.get_scores()

In [ ]:
sentiment_generation.to_csv('news.csv')

In [ ]:
sentiment_generation.document_file.to_csv('news.csv')

In [ ]:
sentiment_generation.document_file.to_csv('sentiment_scores.csv')

In [ ]:
sentiment_generation.document_file.similarity.head()

In [1]:
import yaml
from langchain.llms import Cohere, OpenAI, AI21
# from langchain import PromptTemplate, LLMChain
# from langchain.callbacks import get_openai_callback
# from langchain.chains import SequentialChain
# import pandas as pd
# import numpy as np
# from datetime import datetime
# from dateutil.relativedelta import relativedelta
# from fuzzywuzzy import fuzz, process
# from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
# from langchain.prompts import FewShotPromptTemplate, PromptTemplate
# from langchain.embeddings import CohereEmbeddings,OpenAIEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.document_loaders import DataFrameLoader,CSVLoader
# import os

with open("apis.yaml", "r") as file:
    yaml_data = yaml.load(file, Loader=yaml.FullLoader)
open_ai_params = {
    "max_tokens": 512,
    "openai_api_key": yaml_data["LLMS"]["OPENAI_API_KEY"],
}
cohere_params = {
    "model": "command-xlarge-nightly",
    "max_tokens": 2202,
    "cohere_api_key": yaml_data["LLMS"]["COHERE_API_KEY"],
    "temperature": 0,
    "k": 0,
}
ai21_params = {
    "model": "j2-jumbo-instruct",
    "numResults": 1,
    "temperature": 0,
    "topP": 1,
    "ai21_api_key": yaml_data["LLMS"]["AI21_API_KEY"],
    "maxTokens": 25,
}
import yaml
from datacreator import StockData,EconomyData,FewShot_Sentiment_Generator

x = FewShot_Sentiment_Generator('AAPL',open_ai_params,cohere_params,ai21_params)

Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

Saving log to /Users/maneelreddy/Downloads/Entreprenuer/lstocklopenbbm/data/logs



In [2]:
x.news_chain_analysis()

Loading faiss.
Successfully loaded faiss.


,ticker,headline,datetime,sentiment_bucket,sentiment_score
0,AAPL,"Apple Earnings May Fall 10% As iPad, Mac Sales...",2023-05-02 11:20:17,Negative,0.38
1,AAPL,"GM Ditches Apple CarPlay In 2024 Chevy Blazer,...",2023-05-02 09:34:18,Negative,0.28
2,AAPL,Apple fights $2 billion London lawsuit for 'th...,2023-05-02 08:34:56,Negative,0.28
3,AAPL,Apple Card savings account has an attractive 4...,2023-05-02 07:25:00,Negative,0.27
4,AAPL,Apple fights $2 bln London lawsuit for 'thrott...,2023-05-02 05:54:58,Negative,0.34
...,...,...,...,...,...
89,AAPL,iPhone 15 to add mysterious new button on the ...,2023-04-28 08:15:25,Neutral,0.50
90,AAPL,Here is What to Know Beyond Why Apple Inc. (AA...,2023-04-28 06:00:01,Neutral,0.50
91,AAPL,"Catalyst watch: FOMC, Apple earnings, Kenvue I...",2023-04-27 19:06:00,Neutral,0.50
92,AAPL,"Apple Stock Is a Winner, but How Much Higher C...",2023-04-27 18:30:00,Neutral,0.50
